In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings


In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
PATH = "/Users/lucaspascal/Library/CloudStorage/OneDrive-Efrei/Projet/my-chatbot-project/code-du-travail.txt"

def read_paragraphs(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        # Assuming paragraphs are separated by two newline characters
        paragraphs = content.split('\n\n')  
        # Trim whitespace and filter out any empty strings
        paragraphs = [p.strip() for p in paragraphs if p.strip() != '']
        return paragraphs

paragraphs = read_paragraphs(PATH)
count =0
for paragraph in paragraphs:
    count += len(paragraph)

mean_letter=count/len(paragraphs)
number_of_paragraphs=len(paragraphs)

646.0670795839663

In [95]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)
   
laws =[]
for paragraph in paragraphs:
    chunk = text_splitter.split_text(paragraph)
    laws.append(chunk)

In [151]:
# EMBEDDING VECTORS

from sentence_transformers import SentenceTransformer
MODEL_EMBEDDING = "dangvantuan/sentence-camembert-large"

# Initialize the model
model = SentenceTransformer(MODEL_EMBEDDING)

No sentence-transformers model found with name /Users/lucaspascal/.cache/torch/sentence_transformers/dangvantuan_sentence-camembert-large. Creating a new one with MEAN pooling.


In [ ]:
# from sentence_transformers import SentenceTransformer
# MODEL_EMBEDDING = "dangvantuan/sentence-camembert-large"

# # Initialize the model
# embedding = SentenceTransformer(MODEL_EMBEDDING)

# # Generate embeddings for the sentences
# for text in texts[:1]:
#     embeddings = model.encode(text.page_content)

In [152]:
# Connect to the Weaviate instance
import weaviate
client = weaviate.Client("http://localhost:8080")

/Users/lucaspascal/Library/CloudStorage/OneDrive-Efrei/Projet/my-chatbot-project/.venv/lib/python3.11/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.4.1.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [153]:
laws[0]

["Tout projet de réforme envisagé par le Gouvernement qui porte sur les relations individuelles et collectives du travail, l'emploi et la formation professionnelle et qui relève du champ de la",
 "négociation nationale et interprofessionnelle fait l'objet d'une concertation préalable avec les organisations syndicales de salariés et d'employeurs représentatives au niveau national et",
 "interprofessionnel en vue de l'ouverture éventuelle d'une telle négociation.",
 "A cet effet, le Gouvernement leur communique un document d'orientation présentant des éléments de diagnostic, les objectifs poursuivis et les principales options.",
 "Lorsqu'elles font connaître leur intention d'engager une telle négociation, les organisations indiquent également au Gouvernement le délai qu'elles estiment nécessaire pour conduire la négociation.",
 "Le présent article n'est pas applicable en cas d'urgence. Lorsque le Gouvernement décide de mettre en oeuvre un projet de réforme en l'absence de procédure de co

In [102]:
# for law_chunks in laws[:2]:
#     for sentence in law_chunks:
#         # Embed each chunk of the law
#         embeddings = model.encode(sentence)
#         print(law_chunks)
#         print(embeddings.tolist())
#         print(type(law_chunks))
#         print(type(embeddings.tolist()))  
#     print('---------------------------')
    
for law_chunks in laws[:2]:
    #law = ' '.join()
    for sentence in law_chunks:
        # Embed each chunk of the law
        embeddings = model.encode(sentence)
        
        # Concatenate the chunks to form the full law        
        print(sentence)
        print(embeddings.tolist())
        

Tout projet de réforme envisagé par le Gouvernement qui porte sur les relations individuelles et collectives du travail, l'emploi et la formation professionnelle et qui relève du champ de la
[-0.28693532943725586, 0.23181487619876862, -0.22361838817596436, -0.12152249366044998, 0.7727704644203186, -0.1863226592540741, 0.22637057304382324, -0.30384090542793274, -0.11752130091190338, -0.08007355779409409, 0.6466899514198303, -0.2978847026824951, 0.03546596318483353, 0.35406753420829773, 0.20128607749938965, 0.32316815853118896, 0.20627573132514954, 0.17204424738883972, 0.466574102640152, 0.18922650814056396, -0.3491799235343933, -0.4575006067752838, -0.09095069020986557, -0.03348522260785103, -0.025764193385839462, 0.39871296286582947, 0.06530933827161789, 0.15989279747009277, 0.2922164797782898, 0.3680247664451599, -0.3101974129676819, 0.2878452241420746, -0.7380471229553223, 0.3570583164691925, -0.023005250841379166, 0.4721175730228424, 0.4112371504306793, 0.8093754649162292, -0.134242

# Embedding

In [ ]:
for law_chunks in laws:
    for sentence in law_chunks:
        # Embed each chunk of the law
        embeddings = model.encode(sentence)
        
        # Concatenate the chunks to form the full law
        #law = ' '.join(law_chunks)
        
        # Create a data object in Weaviate for each law
        data_object = client.data_object.create(
            class_name="Document",
            data_object={
                "content": sentence
            },
            vector=embeddings.tolist()
        )

In [201]:
# QESTION EMBEDDED
question = "Quelle est la durée de période d’essai d’un cadre pour un contrat de travail à durée indéterminé ?"
question_embedded = model.encode(question)
question_embedded

array([ 0.35245395,  0.07604571,  0.45864415, ..., -0.26084882,
       -0.50151086,  0.4575413 ], dtype=float32)

In [210]:
# RESPONSE QUERY FOR THE LLM
response = (
    client.query
    .get('Document', ['content'])
    .with_near_vector({
        "vector": question_embedded
    })
    .with_limit(3)
    .with_additional(["distance"])
    .do()
)

#context = response['data']['Get']['Document'][0]['content']
all_contents = '\n'.join(document['content'] for document in response['data']['Get']['Document'])
print(all_contents)

Le contrat de travail à durée déterminée peut comporter une période d'essai.
Le contrat de travail à durée indéterminée peut comporter une période d'essai dont la durée maximale est :
1° Pour les ouvriers et les employés, de deux mois ;
La durée du contrat de travail à durée déterminée est déduite de la période d'essai éventuellement prévue dans le nouveau contrat de travail.


In [120]:
#QUERY ALL DATA

query = (
client.query.get('Document', ['content']).with_additional(["id vector"])
.with_limit(1)
).do()

In [156]:
# Count the number of documents in Weaviate
response = (
    client.query
    .aggregate("Document")
    .with_meta_count() 
    #.with_fields("vector {count maximum mean median minimum mode sum type}")
    .do()
)
response

{'data': {'Aggregate': {'Document': [{'meta': {'count': 9861}}]}}}

In [ ]:
# Step 1: Retrieve All Object IDs
query = (
    client.query.get('Document', ['content'])
    .with_additional(['id', 'vector'])
    .with_limit(10000)  # Adjust the limit based on your needs
    .do()
)

# Extract the IDs from the query results
object_ids = [item['_additional']['id'] for item in query['data']['Get']['Document']]

# Step 2: Delete Each Object by ID
for obj_id in object_ids:
    try:
        client.data_object.delete(
            class_name="Document",
            uuid=obj_id
        )
        print(f'Successfully deleted object with ID: {obj_id}')
    except Exception as e:
        print(f'Failed to delete object with ID: {obj_id}. Error: {e}')

# LLM

In [157]:
# Run model Llama-ccp

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp

In [158]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
# # Make sure the model path is correct for your system!
# llm = LlamaCpp(
#     model_path="/Users/lucaspascal/Library/CloudStorage/OneDrive-Efrei/Projet/my-chatbot-project/mistral-7b-instruct-v0.1.Q3_K_L.gguf",
#     temperature=0.1,
#     max_tokens=2000,
#     top_p=1,
#     callback_manager=callback_manager,
#     verbose=True,  # Verbose is required to pass to the callback manager
# )

In [191]:
n_gpu_layers = 10  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/Users/lucaspascal/Library/CloudStorage/OneDrive-Efrei/Projet/my-chatbot-project/model/mistral-7b-instruct-v0.1.Q5_K_S.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    streaming=True,
    verbose=True,  # Verbose is required to pass to the callback manager
)

/Users/lucaspascal/Library/CloudStorage/OneDrive-Efrei/Projet/my-chatbot-project/.venv/lib/python3.11/site-packages/langchain_core/language_models/llms.py:182: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  warnings.warn(
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/lucaspascal/Library/CloudStorage/OneDrive-Efrei/Projet/my-chatbot-project/model/mistral-7b-instruct-v0.1.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096

In [193]:
# template = """Tu es un expert en droit, répond en francais à cette question basé sur le contexte suivant:
# "{context}"

# Question: {question}
# """
 
prompt = PromptTemplate(template=template, input_variables=["context", "question"])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

In [194]:
answer = llm_chain.invoke({'context':context,'question':question})



> Entering new LLMChain chain...
Prompt after formatting:
Tu es un expert en droit, répond en francais à cette question basé sur le contexte suivant:
"La forfaitisation de la durée du travail doit faire l'objet de l'accord du salarié et d'une convention individuelle de forfait établie par écrit."

Question: Quelles sont les conditions juridiques pour valider et sécuriser le suivi d’une convention annuelle de forfait en jours travaillés ?


Answer: Pour que la convention annuelle de forfait en jours travaillés soit validée et sécurisée, les conditions suivantes doivent être remplies :

1. L'accord du salarié est obligatoire. Le salarié doit avoir accepté, par écrit, le forfait annuel de travail. Cet accord doit être signé par le salarié et par le employeur ou la représentante légale de l'entreprise.
2. La convention individuelle de forfait doit être établie en écriture. La convention doit être complète, c'est-à-dire qu'elle devrait inclure les dispositions concernant le salaire, la du


llama_print_timings:        load time =    5785.35 ms
llama_print_timings:      sample time =      33.98 ms /   256 runs   (    0.13 ms per token,  7533.62 tokens per second)
llama_print_timings: prompt eval time =    5785.08 ms /   112 tokens (   51.65 ms per token,    19.36 tokens per second)
llama_print_timings:        eval time =   43574.54 ms /   255 runs   (  170.88 ms per token,     5.85 tokens per second)
llama_print_timings:       total time =   50126.56 ms /   367 tokens


In [219]:
def get_answer(context, query, llm):
    template = """
    ### [INST] 
    Instruction: Tu es un assistant utile, respectueux et honnête. 
    Réponds précisément et en peu de mots selon le contexte.

    ### CONTEXTE:
    {context}

    ### QUESTION:
    {query} 

    [/INST]
    """
    prompt = PromptTemplate(template=template, input_variables=["context", "query"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    
    #answer = llm_chain.run({'context':context,'query':query})
    #answer = llm_chain.invoke({'context':context,'query':query})
    return llm_chain.run({'context':context,'query':query})


In [220]:
get_answer(context, question, llm)

Llama.generate: prefix-match hit



    La durée du contrat de travail à durée indéterminée n'est pas précisée, mais une période d'essai peut être ajoutée.


llama_print_timings:        load time =    5785.35 ms
llama_print_timings:      sample time =       5.26 ms /    39 runs   (    0.13 ms per token,  7415.86 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    6027.44 ms /    39 runs   (  154.55 ms per token,     6.47 tokens per second)
llama_print_timings:       total time =    6127.54 ms /    40 tokens


"\n    La durée du contrat de travail à durée indéterminée n'est pas précisée, mais une période d'essai peut être ajoutée."

In [215]:
type(answer)

str